# Import packages and database

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [2]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [3]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [8]:
res.compile_estimation_results(all_results)

(                                         Model0
 Number of estimated parameters                5
 Sample size                                5000
 Final log likelihood               -4587.818071
 Akaike Information Criterion        9185.636142
 Bayesian Information Criterion      9218.222108
 asc_cycle (t-test)                 -3.88  (-36)
 asc_drive (t-test)                -1.3  (-16.1)
 asc_pt (t-test)                  -0.503  (-9.3)
 beta_cost (t-test)              -0.194  (-13.9)
 beta_time (t-test)                -5.5  (-26.3),
 {'Model0': 'Model0'})

In [9]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [10]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [11]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [12]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847226,0.202570,-23.928648,0.0
asc_drive,-1.969875,0.137929,-14.281801,0.0
asc_pt,-2.337198,0.139506,-16.753333,0.0
beta_cost,-0.182009,0.016087,-11.314048,0.0
beta_time_cycle,-5.312452,0.462002,-11.498763,0.0
beta_time_drive,-6.474307,0.378989,-17.083106,0.0
beta_time_pt,-3.538619,0.250656,-14.117426,0.0
beta_time_walk,-8.430774,0.418991,-20.121597,0.0


In [13]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1
 Number of estimated parameters                5                8
 Sample size                                5000             5000
 Final log likelihood               -4587.818071     -4275.496542
 Akaike Information Criterion        9185.636142      8566.993084
 Bayesian Information Criterion      9218.222108       8619.13063
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)
 beta_time (t-test)                -5.5  (-26.3)                 
 beta_time_cycle (t-test)                          -5.31  (-11.5)
 beta_time_drive (t-test)                          -6.47  (-17.1)
 beta_time_pt (t-test)                             -3.54  (-14.1)
 beta_time_walk (t-test)                           -8.43  (-20.1),
 {'Model0

In [14]:
biogeme_model1.calculateNullLoglikelihood(av)

-6931.471805599917

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [15]:
old  = results_generic
new  = results_specific_time
new.likelihood_ratio_test(old, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=624.6430583662859, threshold=11.344866730144373)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [16]:
import biogeme.segmentation as seg
gender_segmentation = seg.DiscreteSegmentationTuple(
    variable=female, mapping={1: 'female', 0: 'male'}
)

We will study its interaction on the ASC

In [17]:
asc_cycle = Beta('asc_walk', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(
    asc_cycle,
    [
        gender_segmentation,
    ],
).segmented_beta()
asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(
    asc_pt,
    [
        gender_segmentation,
    ],
).segmented_beta()
asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(
    asc_drive,
    [
        gender_segmentation,
    ],
).segmented_beta()

And its interaction on the cost attribute in the car and public transport alternative.

In [18]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(
    beta_cost, [gender_segmentation]
).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [19]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [20]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_drive,-1.892866,0.144171,-13.129307,0.000000e+00
asc_drive_male,-0.199634,0.096428,-2.070288,3.842534e-02
asc_pt,-2.206895,0.146753,-15.038152,0.000000e+00
asc_pt_male,-0.336024,0.103094,-3.259401,1.116478e-03
asc_walk,-5.456936,0.254321,-21.456875,0.000000e+00
asc_walk_male,1.054026,0.209600,5.028748,4.936922e-07
beta_cost,-0.184656,0.022353,-8.261019,2.220446e-16
beta_cost_male,0.003562,0.032096,0.110969,9.116406e-01
beta_time_cycle,-5.577830,0.472853,-11.796127,0.000000e+00
beta_time_drive,-6.488269,0.380036,-17.072789,0.000000e+00


In [21]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1  \
 Number of estimated parameters                5                8   
 Sample size                                5000             5000   
 Final log likelihood               -4587.818071     -4275.496542   
 Akaike Information Criterion        9185.636142      8566.993084   
 Bayesian Information Criterion      9218.222108       8619.13063   
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                -5.5  (-26.3)                    
 beta_time_cycle (t-test)                          -5.31  (-11.5)   
 beta_time_drive (t-test)                          -6.47  (-17.1)   
 beta_time_pt (t-test)                             -3.54  (-14.1)   
 beta_time_walk (t-test)          

In [22]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

In [23]:
old  = results_specific_time
new  = results_segmented_gender
new.likelihood_ratio_test(old, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=60.524759989359154, threshold=13.276704135987622)

 ###### Answer to question 1
 


The objective of Model 2 is to introduce a socioeconomic characteristic and analyze its influence on the value function of various transportation alternatives. This approach is valid under specific conditions:

Heterogeneity in Individual Preferences: In Model 2, it is assumed that individuals exhibit heterogeneity in their preferences. This acknowledges that people have diverse tastes and characteristics, leading them to weigh the attributes of transportation modes differently. 

Continuous Socioeconomic Characteristics: The socioeconomic characteristic introduced in the model is assumed to be a continuous variable that captures the variations in preferences among individuals based on their socioeconomic attributes.

##### Answer to question 2

Gender-Specific ASCs:

The gender specific ASCs for male individuals in the context of drive and public transport are negative, suggesting that male individuals have a lower baseline preference for these modes compared to female. However it is positive and rather high for walking, meaning that in similar condition male respondant are more likely to choose to walk compared to the female respondant.

The p-values for these gender-specific ASCs are statistically significant, indicating that gender plays a significant role in mode choice preferences.

Gender-Specific Beta Coefficients:

The beta coefficient for cost among male individuals is positive, but the p-value is not statistically significant. This suggests that the impact of cost on mode choice may not be significantly different for male individuals.


##### Answer to question 3

Conducting a likelihood ratio test to compare Model 1 and Model 0 is  relevant  in this analysis because Model 1 is  an extension of Model 0. 

In this context, the null hypothesis for the likelihood ratio test posits that gender does not have a statistically significant impact on the choice of transportation mode. A rejection of the null hypothesis implies that gender indeed plays a significant role in mode choice.

Based on the results of the likelihood ratio test, we can confidently reject Model 1 at a 1% significance level. This indicates that the inclusion of gender as an explanatory variable significantly improves the model's ability to explain the variations in mode choice behavior among individuals.